---
layout: article
title: Introduction to Index Fund
custom_css: article.css
include_mathjax: true
---

In [ ]:
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.ticker import FormatStrFormatter
from scipy.optimize import minimize

# define watermark 
def add_watermark(ax, x, y):
    ax.text(ax.get_xlim()[0]+ x,
            ax.get_ylim()[0]+ y,
            "ladydragoncapital",
            alpha=0.3, fontsize=16)

# globals
HOME_DIR = '/home/jovyan/_jupyter'
DATA_DIR = os.path.join(HOME_DIR, 'data')

# read in csv data
growth = pd.read_csv(os.path.join(DATA_DIR, 'Historical_Returns_on_Stocks_Bonds_and_Bills_1928_2024.csv'))

## Introduction ##

In the previous article - [How to Invest 2000 Dollars in the Optimal Way? - Part I ](https://littlemissdragon.github.io/blog/2025/01/20/how-to-invest-2000-dollars-in-the-optimal-way), we learned about portfolio diversification through different nations. As a continuation of this series, next we will explore diversification through different asset classes, such as bond, index fund, stock, real estate, etc.. From this article forward, we will introduce one asset class in every article till we have enough knowledge to develop a diversified portfolio with the maximum advantage: low risk and high return.

In this article, we will introduce index funds. We will explore the concept of index funds, their features, types of index funds, valuation, how and when to invest in index funds.

## What is an index fund ##

An index fund is a fund that tracks a market index. A market index is a statistical measure that tracks the performance of a specific group of assets, such as stocks, bonds, or other securities, within a financial market. So an index fund itself serves as a diversified portfolio of a particular segment of the market or the market as a whole. 

For instance, Standard & Poor's 500 (S&P 500) Index is a mainly market-capitalization-weighted index of 500 leading publicly traded companies in the US. Another example, Dow Jones US Small-Cap Index is a stock market index that tracks the performance of small-capitalization (300 million - 2 billion) companies in the United States. 

## Key features ##

Pros:

+ Moderate volatility and returns compared to stocks and bonds.

Due to the diversified portfolio, index funds often have lower risk compared to the stocks of individual companies. Meanwhile, index funds aim to match the performance of the market over time, leading to higher returns than conservative asset classes like government or corporate bonds.

For instance, as shown in Figure 1, we compared the annual growth of S&P 500 index, Microsoft stocks and 10-year treasury bond between 1987 and 2024. It is obvious that its volatility and annual growth rate is between those of the Microsoft stocks and the treasury bond. Moreover, Figure 2 shows the means and standard deviations of each asset class. The average growth rate and standard deviation of S&P 500 are both lower than Microsoft (12.45%±17.01% vs. 31.84%±42.33%, respectively), but both higher than the treasury bond (4.97%±2.30%).


In [ ]:
# visualize the annual growth rate of S&p 500 vs. MSFT
fig_count = 1
msft = growth[59:].copy()

ind = np.arange(len(msft['Year']))  # the y locations for the groups
width = 0.2  # the width of the bars

# set the style to a dark theme
plt.style.use("dark_background")

# match website background
plt.rcParams["figure.facecolor"] = "#181818"
plt.rcParams["axes.facecolor"] = "#181818"
plt.rcParams["axes.edgecolor"] = "#181818"

fig, ax = plt.subplots(figsize=(8, 6))

rects1 = ax.bar(ind + width, msft['S&P 500 (includes dividends) %'], width, label='S&P 500', align='center', color='lime')
rects2 = ax.bar(ind, msft['MSFT Annual Change %'], width, label='Microsoft Stock', align='center', color='salmon')
rects3 = ax.bar(ind - width, msft['US T Bond Interest Rate (10-year) %'], width, label='10-Year Treasury Bond', align='center', color='lightblue')
# add watermark
add_watermark(ax, 2, 10)

# Add some text for labels, title and custom x-axis tick labels, etc.
step = 7
ax.set_xticks(ind[::step], msft['Year'][::step])
ax.legend()

plt.xlabel('Year', fontsize=10)
plt.ylabel('Growth Rate %', fontsize=11)
# set title
plt.suptitle(
    f"Figure {fig_count}. 1987-2024 Annual Growth Rate of S&P 500 Index, Microsoft Stock, and 10-Year Treasury Bond", y=0.0001, fontsize=10
)
fig_count += 1
plt.legend(fontsize=9)
plt.show()

In [ ]:
# plot the mean and std for sp 500 and microsoft
spmean = msft['S&P 500 (includes dividends) %'].mean()
msmean = msft['MSFT Annual Change %'].mean()
tbmean = msft['US T Bond Interest Rate (10-year) %'].mean()
spstd = msft['S&P 500 (includes dividends) %'].std()
msstd = msft['MSFT Annual Change %'].std()
tbstd = msft['US T Bond Interest Rate (10-year) %'].std()

categories = ['S&P 500', 'Microsoft Stocks', 'Treasury Bond']
means = [spmean, msmean, tbmean]
std_devs = [spstd, msstd, tbstd]

# set the style to a dark theme
plt.style.use("dark_background")

# match website background
plt.rcParams["figure.facecolor"] = "#181818"
plt.rcParams["axes.facecolor"] = "#181818"
plt.rcParams["axes.edgecolor"] = "#181818"

fig, ax = plt.subplots(figsize=(8, 6))

width = 0.4
ind = np.arange(len(categories))  # x-axis positions

# Plot the bars
bars = ax.bar(ind, means, width, color='lightblue')

# Add error bars (standard deviation)
ax.errorbar(ind, means, yerr=std_devs, fmt='none', color='yellow')

# Add labels, title, and legend
ax.set_xticks(ind)
ax.set_xticklabels(categories)


plt.xlabel('Asset', fontsize=10)
plt.ylabel('Growth Rate %', fontsize=11)
plt.xlim([-1, 3])
# set title
plt.suptitle(
    f"Figure {fig_count}. 1987-2024 Average Annual Growth Rate of S&P 500 Index, Microsoft Stock, and 10-Year Treasury Bond", y=0.0001, fontsize=10
)
fig_count += 1
plt.show()

Meanwhile, we calculated the investment values of 100 dollars into these three asset classes from 1987 to 2024. Figure 3 shows that the 100-dollar investment into S&P 500 becomes 5345.38 dollars, while 626.95 dollars for the treasury bond, and 406,514.73 dollars for the Microsoft stocks.

In [ ]:
# Calculate 10-year T Bond investment growth
def calculate_treasury_growth(df):
    """
    Calculate the growth of a $100 investment in 10-Year Treasury Bonds from 1928 to 2024.
    
    Parameters:
        df (pd.DataFrame): A DataFrame with columns 'Year' and 'Yield'.
    
    Returns:
        pd.Series: A Series containing the investment value for each year.
    """
    # Initial investment
    bond_investment = 100.0
    sp_investment = 100.0
    msft_investment = 100.0
    
    # List to store investment values for each year
    bond_investment_values = []
    sp_investment_values = []
    msft_investment_values = []

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # calculate investment values for t bond
        bond_rate = row['US T Bond Interest Rate (10-year) %'] / 100  # Convert percentage to decimal
        bond_investment *= (1 + bond_rate)  # Compound annually
        bond_investment_values.append(bond_investment)
        # calculate investment values for sp500
        sp_rate = row['S&P 500 (includes dividends) %'] / 100  
        sp_investment *= (1 + sp_rate)  # Compound annually
        sp_investment_values.append(sp_investment)
        # calculate investment values for microsoft
        msft_rate = row['MSFT Annual Change %'] / 100  
        msft_investment *= (1 + msft_rate)  # Compound annually
        msft_investment_values.append(msft_investment)

    # Add the investment values to the DataFrame
    df.loc[:, '100-Dollar Investment US T Bond (1987)'] = bond_investment_values
    df.loc[:, '100-Dollar Investment S&P 500 (1987)'] = sp_investment_values
    df.loc[:, '100-Dollar Investment MSFT (1987)'] = msft_investment_values
    return df

# Calculate T Bond investment growth
msft = calculate_treasury_growth(msft)

# visualize the growth of 100 dollars investment for S&P 500 and US Small Cap 
x = msft['Year']
v = msft['100-Dollar Investment S&P 500 (1987)']
u = msft['100-Dollar Investment US T Bond (1987)']
m = msft['100-Dollar Investment MSFT (1987)']

# set the style to a dark theme
plt.style.use("dark_background")

# match website background
plt.rcParams["figure.facecolor"] = "#181818"
plt.rcParams["axes.facecolor"] = "#181818"
plt.rcParams["axes.edgecolor"] = "#181818"

fig, ax = plt.subplots(figsize=(8, 6))

plt.plot(x, v, label='100-Dollar Investment S&P 500', color='lime')
plt.plot(x, u, label='100-Dollar Investment US T Bond', color='orange')
plt.plot(x, m, label='100-Dollar Investment MSFT', color='blue')

# add watermark
add_watermark(ax, 1, 1)

# set decimal points for x axis
ax.xaxis.set_major_formatter(FormatStrFormatter('%.0f'))

plt.xticks(rotation=0, fontsize=10)
plt.yticks(fontsize=10)
plt.ylabel('US Dollar', fontsize=11)
plt.xlabel('Year', fontsize=11)
plt.xlim([1986, 2025])

# set title
plt.suptitle(
    f"Figure {fig_count}. 1987-2024 100-Dollar Investment Growth of S&P 500 Index, Microsoft Stock, and 10-Year Treasury Bond", y=0.0001, fontsize=10
)
fig_count += 1
plt.legend()
plt.show()


+ Lower fees than other funds

  index funds are passively managed (i.e. low turnover and trading cost), so they tend to have lower commission fees compared to actively managed funds.
  
+ Transparency

  The holdings of an index fund are based on the index it tracks, making it easy to understand its composition.

+ More tax-friendly than bonds or certificate of deposit

  long term (> 1 year) capital gain from an index fund has lower tax rate than the ordinary income tax, while the interest from bonds or certificate of deposit is taxed as ordinary income. (See Appendix I for more information on taxation.)

+ No guaranteed return.

  Unlike bonds or certificate of deposit that have a fixed interest rate, index funds' capital gain is not guaranteed and in short term the investment may experience loss, as shown in Figure 1. Therefore, the investors have to committee long term to the investment, just like bonds or certificate of deposit also requires maturity.


## What types of index funds to invest ##

Index funds can be categorized based on the type of index they track. Here are the main types:

+ Broad Market Index Funds:

  Track major market indices that represent the overall market, such as S&P 500 Index Funds, Total Stock Market Index Funds.

+ Sector-Specific Index Funds:

  Focus on a specific industry or sector, such as technology, healthcare, or energy. For example, NASDAQ-100 Index Funds (tech-heavy), S&P 500 Energy Sector Index Funds.

+ International Index Funds:

  Track indices of foreign markets or global markets. Examples: MSCI Emerging Markets Index Funds, FTSE 100 Index Funds (UK).

+ Bond Index Funds:

  Track fixed-income indices, such as government or corporate bonds. Examples: Bloomberg U.S. Aggregate Bond Index Funds.

+ Commodity Index Funds:

  Track the performance of commodities like gold, oil, or agricultural products.

+ Dividend Index Funds:

  Focus on indices of companies that pay high dividends, such as Dow Jones U.S. Dividend 100 Index Funds.

+ ESG (Environmental, Social, Governance) Index Funds:

  Track indices of companies that meet specific ESG criteria.

+ Small-Cap, Mid-Cap, and Large-Cap Index Funds:

  Focus on companies of different market capitalization, such as Russell 2000 Index Funds (small-cap), S&P MidCap 400 Index Funds.

+ Custom or Thematic Index Funds:

  Track indices based on specific themes, such as clean energy, artificial intelligence, or cybersecurity.


## Investment vehicles: mutual funds or exchange-traded funds ##

Index funds mainly have two investment vehicles (products that investors use): mutual funds and exchange-traded funds (ETFs). Both are pooled investment vehicles that collect money from multiple investors to invest in index funds, but they also have the following differences:

+ Mutual Funds:

  + Investors buy or sell shares directly from the fund company.
  + Priced once per day at the Net Asset Value (NAV).
  + Higher fees than index ETFs.
  + Often require a minimum investment requirement, such as 1000 dollars.
  + Less tax-efficient due to the potential for frequent capital gains distributions. (See details in Appendix II)

+ Exchange-Traded Funds (ETFs):

  + Trade on stock exchanges like individual stocks.
  + Priced continuously throughout the trading day, and the price may deviate slightly from the NAV due to market forces.
  + Lower fees than index mutual funds
  + No or very low minimum investment requirement, such as 10 dollars.
  + More tax-efficient due to the in-kind creation/redemption mechanism. (See details in Appendix II)

For new investors like us, index ETFs are more advantageous than mutual funds considering its lower fees, no minimum investment requirement, trading flexibility, and tax efficiency.



## Valuation of index funds ##

By the valuation of index funds, we can determine whether they are overvalued or undervalued. Here are the factors we can consider:

+ P/E ratio
  P/E ratio (price-to-earning ratio) measures the relationship between a company's stock price and its earnings per share (EPS). For an index fund, which tracks a specific market index (like the S&P 500), the P/E ratio provides insight into the overall valuation of the stocks within the index. By comparing the current P/E ratio of the index fund with its historical average, and considering other factors such as economic conditions, we can determine if the fund is overvalued or undervalued.
+ P/B value
  P/B ratio (price-to-book ratio) provides insight into the overall valuation of the stocks within the index relative to their net asset value. It is particularly useful for evaluating value-oriented investments such as utility companies. Book value (total net asset) is often considered a "floor" value for a company, as it reflects the theoretical liquidation value. A P/B ratio less than 1 suggests the stock may be undervalued (trading below its book value). Meanwhile, we can compare the current P/B ratio of the index fund with its historical average to determine if the market is overvalued or undervalued.

## Valuation of index funds ##

The valuation of the S&P 500 index fund depends on several factors. Here are the most important ones:

+ Earnings-Based Metrics:

  + Earnings Per Share (EPS): Represents the profitability of the companies in the index.

  + Price-to-Earnings (P/E) Ratio: Reflects the market's valuation of earnings.

+ Book Value-Based Metrics:

  + Book Value Per Share: Represents the net asset value of the companies.

  + Price-to-Book (P/B) Ratio: Reflects the market's valuation of net assets.

+ Dividend Metrics:

  + Dividend Yield: Represents the income generated by the index fund relative to its price.

+ Macroeconomic Factors:

  + Interest Rates: Lower interest rates typically support higher valuations.

  + Inflation: High inflation can erode earnings and reduce valuations.

  + GDP Growth: Strong economic growth supports higher earnings and valuations.

+ Market Sentiment:

  + Volatility Index (VIX): Reflects investor fear or complacency.

  + Investor Confidence: Measured through surveys or market activity.

+ Historical Averages:

  + Historical P/E, P/B, and dividend yield averages for the S&P 500.



## How and when to invest ##

To be explored in the future.


## Appendices ##

Appendix I. Tax for capital gain

+ Long-term (> 1 year) capital gain:
  + 0% for 0-44,625 dollars;
  + 15% for 44,626-492,300;
  + 20% for >492,300.

+ Short-term (< 1 year) capital gain:
  Same as ordinary income.

+ Ordinary income (progressive tax system):
  + 10% for 0-11,000 dollars;
  + 12% for 11,001-44,725;
  + 22% for 44,726-95,375;
  + 35% for 231,251-578,125;
  + 37% for > 578,126.

Appendix II. Taxation differences between index mutual funds and ETFs

+ Capital Gains Distributions

    + Index Mutual Funds:
    
        + When investors redeem shares, the fund may need to sell securities to raise cash, which can trigger capital gains.
        
        + These gains are distributed to all shareholders, who must pay taxes on them, even if they did not sell their shares.
        
        + Example: If a mutual fund sells appreciated stocks to meet redemptions, shareholders receive a capital gains distribution at the end of the year.
    
    + Index ETFs:
    
        + ETFs use an in-kind creation/redemption process, which minimizes the need to sell securities.
        
        + When investors redeem ETF shares, the fund delivers a basket of securities (instead of cash) to authorized participants (APs), avoiding taxable events.
        
        + As a result, ETFs typically generate fewer capital gains distributions compared to mutual funds.

+ Tax Efficiency

    + Index Mutual Funds:

        + Less tax-efficient due to the potential for frequent capital gains distributions.

        + Even index mutual funds, which have lower turnover than actively managed funds, can still generate taxable events from rebalancing or meeting redemptions.

    + Index ETFs:

        + More tax-efficient because of the in-kind creation/redemption mechanism.

        + This structure allows ETFs to avoid realizing capital gains, making them ideal for taxable accounts.

## References: ##

+ https://macrotrends.net/stocks/charts/MSFT/microsoft/stock-price-history
+ https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/histretSP.html